In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df_dispo = pd.read_csv('DisponibiliteAlimentaire_2017.csv')
df_pop = pd.read_csv('Population_2000_2018.csv')

In [ ]:
produit_cible = 'Blé et produits'
poids = {
    'taux_dependance': 0.5, 
    'dispo_par_hab': 0.2,   
    'population': 0.3     
}

In [ ]:
df_pop_2017 = df_pop[df_pop['Année'] == 2017].copy()
df_pop_2017['Population'] = df_pop_2017['Valeur'] * 1000
df_pop_2017 = df_pop_2017[['Zone', 'Population']]
df_pop_2017.head()

In [ ]:
elements_utiles = [
    'Production', 
    'Importations - Quantité', 
    'Exportations - Quantité', 
    'Disponibilité alimentaire en quantité (kg/personne/an)',
    'Disponibilité intérieure'
]
df_target = df_dispo[df_dispo['Produit'] == produit_cible].copy()
df_target = df_target[df_target['Élément'].isin(elements_utiles)]

In [ ]:
df_pivot = df_target.pivot_table(
    index= ['Zone', 'Produit'],
    columns='Élément',
    values='Valeur',
    aggfunc='sum'
).reset_index()
df_pivot.head()

In [ ]:
df_final = pd.merge(df_pivot, df_pop_2017, on='Zone', how='inner')
df_final.fillna(0, inplace=True)
colonnes_renommees = {
    'Zone': 'Pays',
    'Importations - Quantité': 'Imports',
    'Exportations - Quantité': 'Exports',
    'Production': 'Production',
    'Disponibilité alimentaire en quantité (kg/personne/an)': 'Dispo_Par_Habitant',
    'Population': 'Population',
    'Disponibilité intérieure': 'Dispo_Interieure'
}
df_final = df_final.rename(columns=colonnes_renommees)
df_final['Taux_Dependance'] = df_final['Imports'] / df_final['Dispo_Interieure']
df_final['Taux_Dependance'] = df_final['Taux_Dependance'].clip(lower=0, upper=1) # << Optionnel

### Indicateur : Consommation(disponibilité) par habitant

In [ ]:
df_final[df_final['Dispo_Par_Habitant']<=10]

In [ ]:
 # Dispo_Par_Habitant entre [0,10] interet
#> 10   ,  pas d'interet 

df_interet = df_final[df_final['Dispo_Par_Habitant']<=10]
df_interet.sort_values('Population', ascending = False)

### Indicateur : Taux_Dependance

In [ ]:
df_interet_dep = df_final[df_final['Taux_Dependance']>0.9]
df_interet_dep.sort_values('Population', ascending = False)

### Normalisation et Classement Final

In [ ]:
scaler = MinMaxScaler()
metrics = ['Taux_Dependance', 'Dispo_Par_Habitant', 'Population']
df_scaled = pd.DataFrame(scaler.fit_transform(df_final[metrics]), columns=metrics)

In [ ]:
df_final = df_final.reset_index(drop=True)
df_final['Score_Final'] = (
    df_scaled['Taux_Dependance'] * poids['taux_dependance'] +
    df_scaled['Dispo_Par_Habitant'] * poids['dispo_par_hab'] +
    df_scaled['Population'] * poids['population']
)

In [ ]:
pd.options.display.float_format = '{:,.0f}'.format   # ou {:,.2f} pour 2 décimales
pd.options.display.float_format = lambda x: f"{x:,.0f}".replace(",", " ")

In [ ]:
df_results = df_final.sort_values(by='Score_Final', ascending=False)
cols_show = ['Pays', 'Produit', 'Taux_Dependance', 'Dispo_Par_Habitant', 'Population', 'Score_Final']

print("\n" + "="*60)
print(f"CLASSEMENT DES PAYS CIBLES POUR : {produit_cible}")
print("="*60)

print(df_results[cols_show].head(10).to_string(formatters={
    'Taux_Dependance': '{:,.1%}'.format,        
    'Dispo_Par_Habitant': '{:,.2f} kg'.format, 
    'Population': '{:,.0f}'.format,            
    'Score_Final': '{:,.4f}'.format         
}, index=False))

In [ ]:
df_results.head()

In [ ]:
df_results.to_csv(f'Resultats_Strategie_{produit_cible}.csv', index=False, sep=';', encoding='utf-8-sig')